In [1]:
import torch
import numpy as np

In [4]:
results = torch.load("eval_convrnn-mc-825-results.pt")

In [7]:
results[0].size(), results[1].size()

(torch.Size([110022, 203]), torch.Size([110022]))

In [9]:
from sklearn.ensemble import RandomForestClassifier

In [115]:
classifier = RandomForestClassifier(n_estimators=15, max_depth=10, random_state=0, class_weight={0: 1, 1:12})

In [26]:
results[1][results[1] != 202] = 0

In [27]:
results[1][results[1] == 202] = 1

In [28]:
y = results[1].numpy()

In [29]:
X = results[0].numpy()

In [55]:
classifier.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=8, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [56]:
# Training accuracy
(classifier.predict(X) == y).mean()

0.9562360255221682

In [119]:
perm = np.random.permutation(X.shape[0])

In [120]:
X = X[perm]
y = y[perm]

In [121]:
split_idx = X.shape[0] // 10
classifier.fit(X[split_idx:], y[split_idx:])
(classifier.predict(X[split_idx:]) == y[split_idx:]).mean()

0.8843062007675218

In [122]:
# Validation set accuracy
print((classifier.predict(X[:split_idx]) == y[:split_idx]).mean())
# Training set FAR/FRR
print(1 - (classifier.predict(X[split_idx:][y[split_idx:] == 1]) == 1).mean())
print(1 - (classifier.predict(X[split_idx:][y[split_idx:] == 0]) == 0).mean())

0.8838393019451009
0.007691666898064553
0.17742473058549046


In [118]:
# Comparison to max accuracy
print(((X[:split_idx].argmax(1) == 202) == y[:split_idx]).mean())
print("Global adaptive threshold training accuracy: {far: 0.008071, frr: 0.1862")

0.9538265769860026
Global adaptive threshold training accuracy: {far: 0.008071, frr: 0.1862


In [111]:
# Validation set FAR
1 - (classifier.predict(X[:split_idx][y[:split_idx] == 1]) == 1).mean()

0.008514901076884573

In [112]:
# Validation set FRR
1 - (classifier.predict(X[:split_idx][y[:split_idx] == 0]) == 0).mean()

0.21857611642174346